# <font color=blue><div align="center">TD-TP 4 - EXERCICE 2</div></font>

### <font color=blue><div align="center">15-02-2022</div></font>

## Modules

In [ ]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Les Données

In [ ]:
nombre_de_sommets = 8
Sommets = range(nombre_de_sommets)

# C1 : dict[(int, int) : int] 
## C1[(i, j)] représente le coût de l'arc (i, j) sur le critère 1
C1 = {(0, 1) : 4,
      (0, 2) : 5,
      (0, 3) : 9,
      (1, 4) : 5,
      (1, 5) : 3,
      (2, 4) : 8,
      (2, 5) : 6,
      (2, 6) : 5,
      (3, 5) : 6,
      (3, 6) : 7,
      (4, 7) : 7,
      (5, 7) : 3,
      (6, 7) : 3
     }

# C2 : dict[(int, int) : int] 
## C2[(i, j)] représente le coût de l'arc (i, j) sur le critère 2
C2 = {(0, 1) : 7,
      (0, 2) : 8,
      (0, 3) : 4,
      (1, 4) : 7,
      (1, 5) : 9,
      (2, 4) : 3,
      (2, 5) : 5,
      (2, 6) : 5,
      (3, 5) : 4,
      (3, 6) : 7,
      (4, 7) : 1,
      (5, 7) : 4,
      (6, 7) : 3
     }

# critereAOptimiser : int
critereAOptimiser = 1      # vaut 1 ou 2        

assert critereAOptimiser == 1 or critereAOptimiser == 2

# C, C_prim : dict[(int, int) : int]
## C et C_prim sont des variables globales permettant de référencer le dictionnaire
### C1 ou C2 en fonction de la valeur attribuée à la variable critereAOptimiser
C = C_prim = None

if critereAOptimiser == 1:
    C = C1
    C_prim = C2
else:
    C = C2
    C_prim = C1

assert C is not None
assert C_prim is not None

# Question 4

In [ ]:
def generer_PCC_model():
    """None -> Model
    instancie et retourne le modèle Gurobi correspondant au Programme Linéaire calculant
    le Plus Court Chemin du sommet 0 au sommet 7. Il utilisera les variables globales C et 
    C_prim définies précédemment."""
    
    # -- Initialisation du modèle --
    # m : Model
    m = Model("TD4-Exo2")
    
    # -- Ajout des variables  --
    # X: dict[(int : int) : Var]
    X = {(i, j) :  m.addVar(vtype = GRB.BINARY, name = f'x_{i}_{j}') for i, j in C}
    # -
    
    # -- Ajout des contraintes --
    # ConstrVertex0 : Constr
    ConstrVertex0 = m.addConstr(quicksum([X[(0, j)] for i, j in C if i == min(Sommets)]) == 1, name='ConstrVertex0')

    # ConstrOtherVertexDict : Dict[int : Constr]
    ConstrOtherVertexDict = {i : m.addConstr(quicksum([X[k, i] for k, l in C if l == i]) == quicksum([X[i, j] for l, j in C if l == i]), name=f'constr{i}') for i in Sommets if i != min(Sommets) and i != max(Sommets)}
    # -
    
    # -- Ajout de la fonction objectif --
    m.setObjective(quicksum([C[(i, j)]*x_i_j for (i, j), x_i_j in X.items()]), GRB.MINIMIZE)
    # -
    
    # -- Choix d'un paramétrage d'affichage minimaliste --
    m.params.outputflag = 0 # mode muet
    # - 

    # -- Mise à jour du modèle  --
    m.update()
    # -
    
    return m
    
    
    

In [ ]:
m = generer_PCC_model()

# -- Affichage en mode texte du PL --
m.display()
#- 

# -- Résolution --
m.optimize()

# -- Vérification du statut et Affichage (le cas échéant) des solutions --
if m.status == GRB.INFEASIBLE:
    print("\n LE PROGRAMME N'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print("\n LE PROGRAMME EST NON BORNÉ!!!")
else:
    # valeur_PCC : int          
    ## la valeur du plus court chemin du sommet 0 au sommet 7.
    valeur_PCC = int(m.objVal)                      # valeur_PCC à compléter  
    print("\n\nLe chemin le plus court a pour longueur : {}".format(valeur_PCC))
    
    # PCC : str
    ## La chaine correspondant au plus court chemin.
    PCC = ""                
    Path = list()
    current_vertex = min(Sommets)
    while current_vertex != max(Sommets):
        Path.append(str(current_vertex))
        current_vertex = max([j for j in Sommets if (current_vertex, j) in C], 
                             key=lambda j : m.getVarByName(f'x_{current_vertex}_{j}').x)
    Path.append(str(current_vertex))
    PCC = "->".join(Path)
    print("Il s'agit de : \n", PCC) 

# Question 6

In [ ]:
# epsilon : float
epsilon = 0.00001

m = generer_PCC_model()

# -- Résolution --
m.optimize()

# otherObjExpr : LinExpr
## represente l'expression lineaire de la fonction objectif correspondant au critère non optimisé (C_prim)
otherObjExpr = quicksum([cout_i_j*m.getVarByName(f'x_{i}_{j}') for (i, j), cout_i_j in C_prim.items()])

it = 0
while m.status != GRB.INFEASIBLE:
    if critereAOptimiser == 1:
        sol = (m.objVal, otherObjExpr.getValue())
    else:
        sol = (otherObjExpr.getValue(), m.objVal)
    print("La solution trouvée à l'itération {} est : {}".format(it, sol))
    # -- Affichage de la solution dans l'espace de decision
    PCC = ""                
    Path = list()
    current_vertex = min(Sommets)
    while current_vertex != max(Sommets):
        Path.append(str(current_vertex))
        current_vertex = max([j for j in Sommets if (current_vertex, j) in C], 
                             key=lambda j : m.getVarByName(f'x_{current_vertex}_{j}').x)
    Path.append(str(current_vertex))
    PCC = "->".join(Path)
    print("Il s'agit de : \n", PCC) 
    # - 
    it += 1
    # -- Ajout de l'epsilon constraint
    m.addConstr(otherObjExpr <= otherObjExpr.getValue() - epsilon, name=f'epsilon_constraint_{it}')
    # -- Mise à jour du modèle  --
    m.update()
    # -- Résolution --
    m.optimize()
    